In [4]:
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import ftplib
from ftplib import FTP_TLS
import ssl
import pandas as pd
import datetime
import numpy as np

class SmartFTP(FTP_TLS):
        def makepasv(self):
            invalidhost, port = super(SmartFTP, self).makepasv()
            return self.host, port

        def ntransfercmd(self, cmd, rest=None):
            conn, size = ftplib.FTP.ntransfercmd(self, cmd, rest)
            if self._prot_p:
                conn = self.context.wrap_socket(conn,
                                                server_hostname=self.host,
                                                session=self.sock.session)  # this is the fix
            return conn, size

def fromFile(filename, cursor):
        # Open and read the file as a single buffer
        fd = open(filename, 'r')
        sqlFile = fd.read()
        fd.close()

        # all SQL commands (split on ';')
        sqlCommands = sqlFile.split(';')
        for command in sqlCommands:
            print("Executing Command:")
            print(command)
            print("***************************************************************************************")
            print("\n\n")
            cursor.execute(command)
def downloadDaily(ftp, filename):
    locations = '/home/elliott/Development/files/{}'.format(filename)
    ftp.retrbinary("RETR " + filename, open(locations,"wb").write, 1024)
def downloadETB(ftp, filename):
    locations = '/home/elliott/Development/files/etb.txt'
    ftp.retrbinary("RETR " + filename, open(locations,"wb").write, 1024)
def downloadPosition(ftp, filename):
    # Datetime
    d = datetime.datetime.combine(datetime.date.today(), datetime.time()) - datetime.timedelta(days=1)
    positionsFileName = 'Scalp_Trade_EXT011_' + d.strftime('%Y%m%d') + ".csv"

    locations = '/home/elliott/Development/files/{}'.format(positionsFileName)
    print(locations)
    ftp.retrbinary("RETR " + filename, open(locations,"wb").write, 1024)
def get_ftp():
    filelist=[]
    ftp = SmartFTP()
    ftp.ssl_version = ssl.PROTOCOL_SSLv23
    ftp.connect('ftp.tradingblock.com', 21, 100)
    ftp.auth()
    ftp.prot_p()
    ftp.login(user='tbftpuser02', passwd='Tr@d1ngBfTp85296')
    ftp.prot_p()
    ftp.cwd("/")
    ftp.dir('-t',filelist.append)

    countDaily = 0
    countPosition = 0
    countETB = 0

    for latest in filelist:
        if "DailyActivityLog_AOS_TBPRO3_" in latest and countDaily == 0:
            splitUp = latest.split()
            print(splitUp[8])
            countDaily = countDaily + 1
            downloadDaily(ftp, splitUp[8])
        if "Scalp_Trade_EXT011_" in latest and countPosition == 0:
            splitUp = latest.split()
            print(splitUp[8])
            countPosition = countPosition + 1 
            downloadPosition(ftp,splitUp[8])
        if "Scalp_Trade_EXT001_" in latest and countETB == 0:
            splitUp = latest.split()
            print(splitUp[8])
            countETB = countETB + 1
            downloadETB(ftp,splitUp[8])

    ftp.quit()
def run():
    d = datetime.datetime.today().strftime('%Y%m%d')
    foundP = False
    ftpFileName = f'Scalp_Trade_EXT001_{d}.csv'
    positionsFileName = f'Scalp_Trade_EXT011_{d}csv'
    yesterday = datetime.datetime.combine(datetime.date.today(), datetime.time()) - datetime.timedelta(days=1)
    yesterday.strftime('%Y%m%d')
    dailyAct = f"DailyActivityLog_AOS_TBPRO3_{yesterday}.csv"
    get_ftp()

run()




Scalp_Trade_EXT011_20230913.csv
/home/elliott/Development/files/Scalp_Trade_EXT011_20230912.csv
Scalp_Trade_EXT001_20230913.csv
DailyActivityLog_AOS_TBPRO3_20230912.csv


In [ ]:
import pysftp
import datetime
from dateutil.relativedelta import relativedelta

host = 'sftp.rqdclearing.com'
port = 22
username = 'tblk\Scalp'
password= 'n@io7n434nyLtsJp'
cnopts = pysftp.CnOpts()
cnopts.hostkeys = None

conn = pysftp.Connection(host=host,port=port,username=username, password=password, cnopts=cnopts)

d = datetime.datetime.today().strftime('%Y%m%d')
yesterday = (datetime.datetime.now() - relativedelta(days = 1)).strftime('%Y%m%d')

def get_cntpty(yesterday, conn):
    try:
        print("connection established successfully")
        with conn.cd('/Contraparty'):
            conn.get(f'TBLK_ScalpContrapartyReport_{yesterday}.csv', f'/tmp/ScalpContrapartyReport_{yesterday}.csv')
    except:
        print('failed to get Contraparty file...')

def get_tblk(d, conn, yesterday):
    try:
        print("connection established successfully")
        with conn.cd('/'):
            conn.get(f'TBLK_Scalp_SODPositions_EXT011_{d}.csv', f'/tmp/trading/positions/TBLK_Scalp_SODPositions_EXT011_{yesterday}.csv')
            conn.get(f'TBLK_Scalp_ExerciseAssignment_{yesterday}.csv', f'/tmp/trading/assignmentFile/TBLK_Scalp_ExerciseAssignment_{yesterday}.csv')
    except:
        print('failed to get TBLK file...')

def run():
    get_cntpty(yesterday, conn)
    get_tblk(d, conn, yesterday)

run()



: 